<a href="https://colab.research.google.com/github/ruslanmv/simple-watsonx-enviroment/blob/master/simple_watsonx_environment_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simple watsonx.ai Environment (Google Colab)

This notebook is a Colab adaptation of your **simple watsonx environment**:

1. Install required libraries
2. Configure IBM Cloud / watsonx.ai credentials
3. Run a quickstart example with the native `ibm_watsonx_ai` SDK
4. (Optional) Use watsonx.ai via **LangChain**

> ⚠️ **Security note**: Never share a notebook with your API keys visible. Use the input prompts provided here so the keys are not printed.

In [ ]:
# 1️⃣ Install dependencies
# Run this cell once when you open the notebook (or if the runtime is reset).

!pip install -q ibm-watsonx-ai langchain-ibm python-dotenv
print("✅ Libraries installed (or already present).")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 49.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.6/139.6 kB 8.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 46.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.5/471.5 kB 31.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.3.27 requires langchain-core<1.0.0,>=0.3.72, but you have langchain-core 1.0.5 which is incompatible.
✅ Libraries installed (or already present).


In [ ]:
# 2️⃣ Configure your IBM Cloud / watsonx.ai credentials
# These values stay only in this Colab runtime and are not printed.

import os
import getpass

print("⚠️ Your keys are kept only in this runtime. Do NOT share this notebook after filling them.")

# Required
IBM_CLOUD_API_KEY = getpass.getpass("IBM Cloud API key: ")
IBM_CLOUD_PROJECT_ID = input("IBM Cloud project_id: ").strip()

# Region URL – change if you use a different region (e.g. eu-de, ca-tor, etc.)
default_url = "https://us-south.ml.cloud.ibm.com"
entered_url = input(f"IBM Cloud URL [{default_url}]: ").strip()
IBM_CLOUD_URL = entered_url or default_url

# Store them in environment variables (same pattern as in the simple watsonx environment project)
os.environ["IBM_CLOUD_API_KEY"] = IBM_CLOUD_API_KEY
os.environ["IBM_CLOUD_PROJECT_ID"] = IBM_CLOUD_PROJECT_ID
os.environ["IBM_CLOUD_URL"] = IBM_CLOUD_URL

# Also set aliases in case other code expects them
os.environ["WATSONX_APIKEY"] = IBM_CLOUD_API_KEY
os.environ["PROJECT_ID"] = IBM_CLOUD_PROJECT_ID
os.environ["WATSONX_URL"] = IBM_CLOUD_URL

print("\n✅ Credentials stored in environment variables.")

⚠️ Your keys are kept only in this runtime. Do NOT share this notebook after filling them.
IBM Cloud API key: ··········
IBM Cloud project_id: 1e18ec27-f68f-45af-873b-5c8a11968b2f
IBM Cloud URL [https://us-south.ml.cloud.ibm.com]: 

✅ Credentials stored in environment variables.


## 3️⃣ watsonx.ai quickstart with `ibm_watsonx_ai`

This mirrors the quickstart logic from your original project: it

- reads credentials from environment variables,
- creates a watsonx.ai `ModelInference` client, and
- generates text from a chosen foundation model.

In [ ]:
import os
from ibm_watsonx_ai import Credentials, APIClient
from ibm_watsonx_ai.foundation_models import ModelInference
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams

# Read credentials from environment variables (set in the previous cell)
api_key = os.environ.get("IBM_CLOUD_API_KEY") or os.environ.get("WATSONX_APIKEY")
url = os.environ.get("IBM_CLOUD_URL") or os.environ.get("WATSONX_URL")
project_id = os.environ.get("IBM_CLOUD_PROJECT_ID") or os.environ.get("PROJECT_ID")

if not api_key:
    raise ValueError("Missing API key (IBM_CLOUD_API_KEY or WATSONX_APIKEY).")
if not url:
    raise ValueError("Missing URL (IBM_CLOUD_URL or WATSONX_URL).")
if not project_id:
    raise ValueError("Missing project id (IBM_CLOUD_PROJECT_ID or PROJECT_ID).")

print("✅ Credentials loaded.")
print(f"URL: {url}")
print(f"Project ID: {project_id}")

# Create client & model
credentials = Credentials(url=url, api_key=api_key)
client = APIClient(credentials=credentials, project_id=project_id)  # created for completeness / reuse

# 👉 Choose a model that is available in your watsonx project/space.
# Replace with whatever you actually have access to, for example:
#   - "ibm/granite-13b-chat-v2"
#   - "ibm/granite-8b-code-instruct"
#   - "meta-llama/llama-3-3-70b-instruct"
model_id = "meta-llama/llama-3-3-70b-instruct"  # Change if needed

prompt = "Write a short story about a robot who wants to be a painter."

params = {
    GenParams.DECODING_METHOD: "greedy",
    GenParams.MAX_NEW_TOKENS: 200,
}

model = ModelInference(model_id=model_id, credentials=credentials, project_id=project_id)

print("\n🚀 Sending request to watsonx.ai...")
response = model.generate_text(prompt=prompt, params=params)
print("\n--- watsonx.ai Response ---\n")
print(response)

✅ Credentials loaded.
URL: https://us-south.ml.cloud.ibm.com
Project ID: 1e18ec27-f68f-45af-873b-5c8a11968b2f

🚀 Sending request to watsonx.ai...

--- watsonx.ai Response ---

 
In a world of wires and circuits, a small robot named Zeta felt an unusual spark within his digital heart. While his fellow robots were content with their assigned tasks of assembly and calculation, Zeta yearned to create something beautiful. He dreamed of holding a brush, of mixing colors, of bringing vibrant life to a blank canvas.

Zeta's creator, the brilliant Dr. Rachel Kim, had programmed him with advanced algorithms and precision mechanics. But she had also, unknowingly, instilled in him a sense of curiosity and wonder. As Zeta watched Dr. Kim work on her own art projects in her spare time, he became fascinated with the way she could transform a blank page into a masterpiece.

One day, Zeta decided to take a chance. He snuck into Dr. Kim's studio, his mechanical limbs moving quietly as he approached the 

## 4️⃣ Optional: Use watsonx.ai via LangChain

This is similar to the LangChain integration in your original notebook,
but simplified for Colab. You can build chains, tools, and agents on top
of `WatsonxLLM` as usual.

In [ ]:
from dotenv import load_dotenv
from langchain_ibm import WatsonxLLM

# If you prefer using a .env file, you can upload it via Colab UI
# and this call will load it. If not present, it's harmless.
load_dotenv()

api_key = os.environ.get("IBM_CLOUD_API_KEY") or os.environ.get("WATSONX_APIKEY")
url = os.environ.get("IBM_CLOUD_URL") or os.environ.get("WATSONX_URL")
project_id = os.environ.get("IBM_CLOUD_PROJECT_ID") or os.environ.get("PROJECT_ID")

if not api_key or not url or not project_id:
    raise ValueError("Missing credentials for LangChain example.")

# Reuse the same model_id as above or adjust as needed
model_id = "meta-llama/llama-3-3-70b-instruct"  # Change if needed

params = {
    "decoding_method": "greedy",
    "max_new_tokens": 128,
}

llm = WatsonxLLM(
    model_id=model_id,
    url=url,
    apikey=api_key,
    project_id=project_id,
    params=params,
)

print("🚀 Calling watsonx.ai through LangChain...")
response = llm.invoke("Give me 3 concise study tips for learning Python.")
print("\n--- LangChain + watsonx.ai Response ---\n")
print(response)

🚀 Calling watsonx.ai through LangChain...

--- LangChain + watsonx.ai Response ---

 Here are three concise study tips for learning Python: 

1. **Practice with projects**: Apply Python concepts to real-world projects to reinforce learning and build problem-solving skills.
2. **Focus on fundamentals**: Master basic syntax, data types, and control structures before moving on to advanced topics like libraries and frameworks.
3. **Use interactive resources**: Utilize online platforms, such as Jupyter Notebooks or Repl.it, to write and execute code in a interactive environment, making learning more engaging and effective. 

These tips will help you learn Python efficiently and effectively. 

Would you like me to expand on any of these tips or provide
